In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# import all the required data
train_df = pd.read_csv("./data/train.csv")
train_df.head()

C:\Users\krish\AppData\Local\Temp\ipykernel_21416\3118064815.py:2: DtypeWarning: Columns (4,13,14,16,47) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("./data/train.csv")


,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,points_y,position,positionText_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,20025,833,642,51,2,1,1,1,1,9.0,...,3.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,922731975
1,20025,833,642,51,2,1,1,1,1,9.0,...,6.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923172525
2,20025,833,642,51,2,1,1,1,1,9.0,...,6.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923833350
3,20025,833,642,51,2,1,1,1,1,9.0,...,10.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,925195050
4,20025,833,642,51,2,1,1,1,1,9.0,...,10.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,926576775


In [4]:
# dropping columns that are useless
new_train_df = train_df[
    ["result_driver_standing", "points_y", "company", "round",
     "status", "wins", "grand_prix", "year", "points", "positionOrder",
     "resultId", "racerId", "position"]]
new_train_df.head()

finished_train = new_train_df[new_train_df["status"] == "Finished"]
finished_train = finished_train.drop(["status"], axis=1)
finished_train = finished_train.fillna(0)

# encoding all columns
label_encoders = {}
for column in ['company', 'grand_prix']:
    le = LabelEncoder()
    finished_train[column] = le.fit_transform(finished_train[column])
    label_encoders[column] = le

In [5]:
X = finished_train.drop(columns=['position'])
y = finished_train['position']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
base_models = [
    ('rf', RandomForestRegressor(n_estimators=50, max_depth=10, n_jobs=-1, random_state=42)),  # Reduced estimators and max depth
    ('svr', SVR(kernel='linear', C=0.1))  # Simplified SVR model
]

# Define the meta-model
meta_model = RidgeCV()

# Create the Stacking Regressor
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model)

# Train the Stacking Regressor
stacking_regressor.fit(X_train, y_train)

# Predict and evaluate
y_pred_stack = stacking_regressor.predict(X_test)
rmse_stack = mean_squared_error(y_test, y_pred_stack, squared=False)
r2_stack = r2_score(y_test, y_pred_stack)

In [ ]:
print(f"rmse: {rmse_stack}")
print(f"r2: {r2_stack}")